<a href="https://colab.research.google.com/github/caiocesarcosta/AndroidTVappTutorial/blob/master/ai_job_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import tensorflow as tf
import numpy as np
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.preprocessing.sequence import pad_sequences

# --- Carregando os Dados Profissionais ---
with open("dados_profissionais.json", "r") as arquivo:
    dados = json.load(arquivo)

# --- Perguntas e Respostas Elaboradas como um Recrutador ---
perguntas_respostas = {
    # --- Abertura ---
    "Olá! Gostaria de saber mais sobre sua experiência em desenvolvimento Android.": dados["resumo"],
    "Seu currículo me chamou a atenção. Pode me falar um pouco sobre suas principais habilidades?": dados["resumo"],

    # --- Experiência ---
    "Vejo que você trabalhou na {empresa}. O que você fazia lá?".format(empresa=dados["experiencia"][0]["empresa"]):
        "Trabalhei na {empresa} como {cargo} de {inicio} a {fim}. Minhas principais responsabilidades eram {descricao}".format(
            cargo=dados["experiencia"][0]["cargo"],
            empresa=dados["experiencia"][0]["empresa"],
            inicio=dados["experiencia"][0]["periodo"].split(" - ")[0],
            fim=dados["experiencia"][0]["periodo"].split(" - ")[1],
            descricao=dados["experiencia"][0]["descricao"]
        ),
    "Quais foram seus principais desafios e aprendizados na {empresa}?".format(empresa=dados["experiencia"][0]["empresa"]):
        "Um dos maiores desafios na {empresa} foi [descreva um desafio]. Aprendi muito sobre [mencione um aprendizado importante] durante esse período.",
    "Que tipo de projetos você desenvolveu na {empresa}?".format(empresa=dados["experiencia"][0]["empresa"]):
        "Na {empresa}, trabalhei em projetos como [cite exemplos de projetos].",
    # ... adicione perguntas sobre suas outras experiências

    # --- Habilidades ---
    "Seu currículo menciona conhecimentos em {habilidade}. Pode me falar mais sobre sua experiência com isso?".format(habilidade=dados["habilidades"][0]):
        "Tenho experiência com {habilidade} desde [mencione quando começou a usar a habilidade]. Utilizei em projetos como [cite exemplos de projetos] e me considero [nível de proficiência] em {habilidade}.",
    # ... adicione perguntas sobre suas outras habilidades

    # --- Objetivos Profissionais ---
    "O que te motiva a buscar uma nova oportunidade?": "Busco novos desafios e oportunidades de crescimento profissional em um ambiente que valorize [mencione seus valores, ex: inovação, trabalho em equipe, etc.].",
    "Por que você se candidatou a esta vaga?": "Me interessei pela vaga porque [mencione os motivos, ex: desafios, tecnologias, cultura da empresa, etc.].",
    "Onde você se vê daqui a 5 anos?": "Me vejo como um desenvolvedor Android ainda mais experiente, contribuindo para projetos inovadores e desafiadores.",
    # ... adicione mais perguntas sobre seus objetivos

    # --- Projetos ---
    "Me fale sobre um projeto pessoal que você se orgulha.": "Um projeto que me orgulho bastante é [nome do projeto]. [Descreva o projeto e seus resultados.]. Você pode conferir o código-fonte em [link do projeto].",

    # --- Perguntas Comportamentais ---
    "Como você lida com situações de pressão no trabalho?": "Em situações de pressão, procuro manter a calma, priorizar as tarefas e buscar soluções colaborativas com a equipe.",
    "Descreva uma situação em que você teve que aprender uma nova tecnologia rapidamente.": "[Conte uma história real e destaque suas habilidades de aprendizado e adaptação.]",

    # --- Encerramento ---
    "Obrigado pelas informações. Tem alguma pergunta para mim?": "Sim, gostaria de saber mais sobre [faça uma pergunta sobre a empresa ou a vaga]."
}

# --- Pré-processamento dos Dados com BERT ---
perguntas = list(perguntas_respostas.keys())
respostas = list(perguntas_respostas.values())

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

perguntas_tokenizadas = [tokenizer.encode(pergunta, add_special_tokens=True) for pergunta in perguntas]
perguntas_treino = pad_sequences(perguntas_tokenizadas, padding='post', maxlen=128) # Ajuste maxlen se necessário

# --- Criando o Modelo com BERT ---
modelo_bert = TFBertModel.from_pretrained('bert-base-uncased')

# --- Função para Obter Embeddings com BERT ---
def obter_embedding_bert(texto):
    input_ids = tokenizer.encode(texto, add_special_tokens=True)
    input_ids = tf.constant([input_ids])
    outputs = modelo_bert(input_ids)
    return outputs.last_hidden_state[:, 0, :]  # Pega o embedding do token [CLS]

# --- Treinando o Modelo (BERT já está pré-treinado) ---
embeddings_perguntas_treino = np.array([obter_embedding_bert(pergunta).numpy()[0] for pergunta in perguntas])

# --- Função para Encontrar a Resposta Mais Similar ---
def encontrar_resposta(pergunta):
    embedding_pergunta = obter_embedding_bert(pergunta).numpy()[0]
    similaridades = tf.keras.losses.cosine_similarity(embedding_pergunta, embeddings_perguntas_treino).numpy()
    indice_resposta = np.argmax(similaridades)
    return perguntas_respostas[perguntas[indice_resposta]]

# --- Testando o Chatbot ---
while True:
    pergunta = input("Recrutador: ")
    if pergunta == "sair":
        break
    resposta = encontrar_resposta(pergunta)
    print("Você:", resposta)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Recrutador: qual seu nome?
Você: [Conte uma história real e destaque suas habilidades de aprendizado e adaptação.]
Recrutador: Quais foram seus principais desafios e aprendizados na BRQ?
Você: Me interessei pela vaga porque [mencione os motivos, ex: desafios, tecnologias, cultura da empresa, etc.].
